In [ ]:
################################################################################
#
# Script to run an obstacle avoidance example for the TwoPlayerUnicycle4D.
#
################################################################################

import os
import numpy as np
import matplotlib.pyplot as plt

from obstacle_dist_cost import ObstacleDistCost
from car_5d import Car5D
from product_multiplayer_dynamical_system import \
    ProductMultiPlayerDynamicalSystem
from two_player_unicycle_4d import TwoPlayerUnicycle4D
from ilq_solver import ILQSolver
from point import Point
from proximity_cost import ProximityCost
#from obstacle_cost import ObstacleCost
from semiquadratic_cost import SemiquadraticCost
from quadratic_cost import QuadraticCost
from player_cost import PlayerCost
from box_constraint import BoxConstraint
from visualizer import Visualizer
from logger import Logger
from semiquadratic_polyline_cost import SemiquadraticPolylineCost
from quadratic_polyline_cost import QuadraticPolylineCost
import sys
import signal

# General parameters.
TIME_HORIZON = 3.0   # s  #Change back to 5
TIME_RESOLUTION = 0.1 # s
HORIZON_STEPS = int(TIME_HORIZON / TIME_RESOLUTION)
LOG_DIRECTORY = "./logs/two_player_zero_sum/"
MAX_V = 7.0 # m/s

# Create dynamics.
car1 = Car5D(4.0)
dynamics = ProductMultiPlayerDynamicalSystem(
    [car1], T=TIME_RESOLUTION)

# Choose an initial state and control laws.
car1_theta0 = np.pi / 2.2 # 90 degree heading # change back to 2.5!
car1_v0 = 4.0             # 5 m/s initial speed
car1_x0 = np.array([
    [6.5],
    [0.0],
    [car1_theta0],
    [0.0],
    [car1_v0]
])


# theta0 = np.pi / 10
# v0 = 5.0

# theta0 = np.pi / 6
# v0 = 5.0

x0 = car1_x0

P1s = [np.zeros((car1._u_dim, dynamics._x_dim))] * HORIZON_STEPS
alpha1s = [np.zeros((car1._u_dim, 1))] * HORIZON_STEPS

# Create the example environment. It will have a couple of circular obstacles
# laid out like this:
#                           x goal
#
#                      ()
#               ()
#                            ()
#
#          x start
goal = Point(6.5, 35.0)
obstacle_centers = [Point(6.5, 15.0)]
obstacle_radii = [4.5]
car1_position_indices_in_product_state = (0, 1)

goal_cost = ProximityCost(car1_position_indices_in_product_state,
                          point=goal,
                          max_distance=np.inf,
                          apply_after_time=HORIZON_STEPS - 1,
                          name="goal")
#obstacle_costs = [ObstacleCost(
#    position_indices=(0, 1), point=p, max_distance=r,
#    name="obstacle_%f_%f" % (p.x, p.y))
#                  for p, r in zip(obstacle_centers, obstacle_radii)]

obstacle_costs = [ObstacleDistCost(
    position_indices=(0, 1), point=p, max_distance=r,
    name="obstacle_%f_%f" % (p.x, p.y))
                  for p, r in zip(obstacle_centers, obstacle_radii)]

#obstacle_costs = [ObstacleDistCost(car1_position_indices_in_product_state, obstacle_centers, obstacle_radii, name="obstacle")]

w_cost = QuadraticCost(dimension=0, origin=0, name="w_cost")
a_cost = QuadraticCost(dimension=1, origin=0, name="a_cost")

#dvx_cost = QuadraticCost(dimension=0, origin=0, name="dvx_cost")
#dvy_cost = QuadraticCost(dimension=1, origin=0, name="dvy_cost")

# Add light quadratic around original values for theta/v.
v_cost_upper = SemiquadraticCost(
    dimension=3, threshold=MAX_V, oriented_right=True, name="v_cost_upper")
v_cost_lower = SemiquadraticCost(
    dimension=3, threshold=0, oriented_right=False, name="v_cost_lower")

# Polyline
#player1_position_indices_in_product_state = (0, 1)
#car1_polyline = Polyline([Point(6.5, 0.0), Point(6.5, 40.0)])
#car1_polyline_boundary_cost = SemiquadraticPolylineCost(
#    car1_polyline, 1.0, car1_position_indices_in_product_state,
#    "car1_polyline_boundary")
#car1_polyline_cost = QuadraticPolylineCost(
#    car1_polyline, car1_position_indices_in_product_state, "car1_polyline")

OBSTACLE_WEIGHT = 200.0 # HJI: 100
GOAL_WEIGHT = 100.0 # HJI: 400  #Change back to 100
#D_WEIGHT = 1000.0 # HJI: 1000
U_WEIGHT = 10.0 # HJI: 100

# V_WEIGHT = 100.0
V_WEIGHT = 0.1 # HJI: 50

# Build up total costs for both players. This is basically a zero-sum game.
player1_cost = PlayerCost()
player1_cost.add_cost(goal_cost, "x", GOAL_WEIGHT)
#for cost in obstacle_costs:
#    player1_cost.add_cost(cost, "x", OBSTACLE_WEIGHT)

#player1_cost.add_cost(v_cost_upper, "x", V_WEIGHT)
#player1_cost.add_cost(v_cost_lower, "x", V_WEIGHT)
#player1_cost.add_cost(w_cost, 0, U_WEIGHT)
#player1_cost.add_cost(a_cost, 0, U_WEIGHT)

# Visualizer.
visualizer = Visualizer(
    [(0, 1)],
    [goal_cost] + obstacle_costs,
    [".-b"],
    1,
    False,
    plot_lims=[-11, 100, 0, 100])

# Logger.
if not os.path.exists(LOG_DIRECTORY):
    os.makedirs(LOG_DIRECTORY)

path_to_logfile = os.path.join(LOG_DIRECTORY, "unicycle_4d_example.pkl")
if len(sys.argv) > 1:
    path_to_logfile = os.path.join(LOG_DIRECTORY, sys.argv[1])

print("Saving log file to {}...".format(path_to_logfile))
logger = Logger(path_to_logfile)

# Set up ILQSolver.
solver = ILQSolver(dynamics,
                   [player1_cost],
                   x0,
                   [P1s],
                   [alpha1s],
                   0.01,
                   None,
                   logger,
                   visualizer,
                   None,
                  [car1_position_indices_in_product_state, goal, obstacle_centers, obstacle_radii])

def handle_sigint(sig, frame):
    print("SIGINT caught! Saving log data...")
    logger.dump()
    print("...done, exiting!")
    sys.exit(0)

signal.signal(signal.SIGINT, handle_sigint)

solver.run()


#ProximityCost(car1_position_indices_in_product_state, goal, max_distance=np.inf, apply_after_time=HORIZON_STEPS - 1, name="goal")


#ObstacleDistCost(car1_position_indices_in_product_state, point=p, max_distance=r, name="obstacle_%f_%f" % (p.x, p.y)) for p, r in zip(obstacle_centers, obstacle_radii)


#[car1_position_indices_in_product_state, goal, obstacle_centers, obstacle_radii]

In [ ]:
for i in range(10):
    print(i)

In [ ]:
[[[array([[100.1,   0. ],
       [  0. ,   2.1]]), array([[100.1,   0. ],
       [  0. ,   2.1]])

In [ ]:
# This is me trying to find the t*
car1_position_indices = (0,1)
x_index, y_index = car1_position_indices
target_position = ()
target_radius = 2

obstacle_position = ()
obstacle_radius = 2

# This is for the target
hold = 0
hold_new = 0
l = np.zeros((self._horizon, 1))
k_tracker = 0
eps = 0.01
for k in range(self._horizon):
    #print("xs is: ", xs)
    #print("xs[k][0:2] is: ", xs[k][0:2])
    #print("xs[k][5:7] is: ", xs[k][5:7])
    hold_new = target(xs[k])   #np.linalg.norm(xs[k][0:2] - target) + 0.5 * eps * np.linalg.norm(us[0][k])**2
    l[k] = hold_new
    
    if k == 0:
        hold = hold_new #This is the first computed distance (at t=0)
    elif hold_new < hold:
        hold = hold_new
        k_tracker = k
        
        
     
"""
This is for obstacle
"""
g = np.zeros((self._horizon, 1))
hold_obs = 0
hold_new_obs = 0
k_track_obs = 0
for j in range(k_tracker, self._horizon):
    g[j] = ObstacleDistance(xs[j], )
    
    if j == k_tracker:
        hold_obs = g
    elif hold_new_obs < hold_obs:
        hold_obs = hold_new_obs
        k_track_obs = j
        
        
if hold_new[k_tracker] < g[k_track_obs]:
    target_margin_function = True
else:
    target_margin_function = False

In [ ]:

class ObstacleDistance(object):
    def __init__(self, x, position_indices, radius):
        self._x_index, self._y_index = position_indices
        self._radius = radius

    def __call__(self, x, k=0):
        dx = x[self._x_index, 0] - self._point.x
        dy = x[self._y_index, 0] - self._point.y
        
        rel_dis = sqrt(dx*dx + dy*dy) - self._radius
        
        if rel_dis < 0:
            return -rel_dis
        
        return rel_dis

In [ ]:
        # Compute relative distance.
        dx = x[self._x_index, 0] - self._point.x
        dy = x[self._y_index, 0] - self._point.y
        relative_distance = torch.sqrt(dx*dx + dy*dy)
        
        

        return min(relative_distance - self._max_distance, torch.zeros(
            1, 1, requires_grad=True).double())**2

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from cost import Cost
from point import Point

class ObstacleCost(Cost):
    def __init__(self, position_indices, point, max_distance, name=""):
        """
        Initialize with dimension to add cost to and a max distance beyond
        which we impose no additional cost.
        :param position_indices: indices of input corresponding to (x, y)
        :type position_indices: (uint, uint)
        :param point: center of the obstacle from which to compute distance
        :type point: Point
        :param max_distance: maximum value of distance to penalize
        :type threshold: float
        """
        self._x_index, self._y_index = position_indices
        self._point = point
        self._max_distance = max_distance
        super(ObstacleCost, self).__init__(name)

    def __call__(self, x, k=0):
        """
        Evaluate this cost function on the given input state.
        NOTE: `x` should be a column vector.
        :param x: concatenated state of the two systems
        :type x: torch.Tensor
        :return: scalar value of cost
        :rtype: torch.Tensor
        """
        # Compute relative distance.
        dx = x[self._x_index, 0] - self._point.x
        dy = x[self._y_index, 0] - self._point.y
        relative_distance = torch.sqrt(dx*dx + dy*dy)

        return relative_distance - self._max_distance

    def render(self, ax=None):
        """ Render this obstacle on the given axes. """
        circle = plt.Circle(
            (self._point.x, self._point.y), self._max_distance,
            color="r", fill=True, alpha=0.75)
        ax.add_artist(circle)
        ax.text(self._point.x - 1.25, self._point.y - 1.25, "obs", fontsize=8)

In [ ]:
import numpy as np
a = np.zeros((4,1))
b = [5]
for k in range(4):
    a[k] = k
if a[0] == 112:
    Bool = True
else:
    Bool = False
    
pos_indices = (0,1)
x_index, y_index = pos_indices
print(x_index + y_index)
    
#print(Bool)
#print(b)

In [ ]:
def TargetDistance(x, position_indices, target_position, target_radius):
    x_index, y_index = position_indices
    dx = x[x_index, 0] - target_position[0]
    dy = x[y_index, 0] - target_position[1]
    
    relative_distance = m.sqrt(dx*dx + dy*dy)
    
    return -(relative_distance - target_radius)

In [ ]:
def ObstacleDistance(x, position_indices, obstacle_position, obstacle_radius):
    x_index, y_index = position_indices
    dx = x[x_idex, 0] - obstacle_position[0]
    dy = x[y_index, 0] - obstacle_position[1]
    
    relative_distance = m.sqrt(dx*dx + dy*dy)
    
    return relative_distance - target_radius